<a href="https://colab.research.google.com/github/Du-nara/ME421-Mechanical-Systems-Lab-A3/blob/main/Vibration/E_20_388_Vibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Week #1**

### **Activity #1**

The TecQuipment vibration analyser consists of:



* A rigid beam of length L
* Left end: pinned support (no vertical translation, free rotation)
* Right end: connected to the frame through a linear spring of stiffness k
* An unbalanced motor mounted at the mid-span of the beam
* Small-amplitude vibrations about the static equilibrium position


The beam is assumed to be:

* Rigid
* Of mass $\ m_b $
* With mass moment of inertia $I_b$ about its center of mass

The motor has:
* Mass $m_m$
* Produces a harmonic excitation force due to imbalance

To capture the first two dominant modes, we select:
1. $q_1$​(t)=θ(t)  : Small rotation of the rigid beam about the pinned end
2. $q_2$​(t)=y(t)  : Vertical displacement of the motor at mid-span

These two coordinates represent:

* A global rotational mode

* A local translational mode due to the motor inertia

Thus, the system is modeled as a 2-DOF system.
	​

For small angular motion:
* Vertical displacement at the right end:  
$y_R$​=Lθ	​
* Vertical displacement at the motor location (mid-span):
$y_M$​=$\frac{L}{2}$ ​θ+y

(a) Beam kinetic energy

The beam undergoes pure rotation about the pinned end:
$ T_b = \frac{1}{2} I_O \dot{\theta}^2 $

Using parallel-axis theorem:
$I_O = I_b + m_b \left(\frac{L}{2}\right)^2$

(b) Motor kinetic energy

The motor undergoes vertical translation:

$T_m = \frac{1}{2}\, m_m \left( \dot{y}_M \right)^2$

$\dot{y}_M = \frac{L}{2}\,\dot{\theta} + \dot{y}$

$T_m = \frac{1}{2}\, m_m \left( \frac{L}{2}\,\dot{\theta} + \dot{y} \right)^2$

(c) Total kinetic energy

$T = T_b + T_m$

Potential Energy of the System;

Only the spring stores potential energy.

Spring extension at the right end:
δ=Lθ

Spring potential energy:
$V = \frac{1}{2}\,k\,(L\theta)^2$

Lagrange’s Equations;

Using generalized coordinates

$q_1 = \theta, \qquad q_2 = y$

$\frac{d}{dt}\!\left( \frac{\partial T}{\partial \dot{q}_i} \right)
- \frac{\partial T}{\partial q_i}
+ \frac{\partial V}{\partial q_i}
= Q_i$


The unbalanced motor produces a harmonic force:

$F(t) = m_e e \,\omega^{2}\sin(\omega t)$

This force acts vertically at the motor location, contributing only to y:

$Q_{\theta} = 0$

$Q_{y} = F(t)$

After applying Lagrange’s equations:

$\begin{bmatrix} M_{11} & M_{12} \\ M_{21} & M_{22} \end{bmatrix} \begin{bmatrix} \ddot{\theta} \\ \ddot{y} \end{bmatrix} + \begin{bmatrix} K_{11} & 0 \\ 0 & 0 \end{bmatrix} \begin{bmatrix} \theta \\ y \end{bmatrix} = \begin{bmatrix} 0 \\ F(t) \end{bmatrix}$

Where:

$M_{11} = I_{0} + m_m (\frac{L}{2})^{2}$

$M_{12} = M_{21} = m_m(\frac{L}{2})$

$M_{22}​=m_m​$

$K_{11}=kL^2$

This 2-DOF model captures the first two dominant vibration modes of the TecQuipment vibration analyser.


## **Activity #2**

From Activity #1, the 2-DOF system is described by:

$
\mathbf{M}
=
\begin{bmatrix}
I_{0} + m_m \left(\dfrac{L}{2}\right)^{2} & m_m \left(\dfrac{L}{2}\right) \\
m_m \left(\dfrac{L}{2}\right) & m_m
\end{bmatrix}
$

$
\mathbf{K}
=
\begin{bmatrix}
k L^{2} & 0 \\
0 & 0
\end{bmatrix}
$

$I_O = I_b + m_b(L/2)^2
$

$m_m$= motor mass,
$𝑚_𝑏$= beam mass,
k = spring stiffness

We assume viscous damping, primarily at the spring/beam supports.

Let’s assume a diagonal damping matrix:

$C=\begin{bmatrix}c_{\theta}&0\\0&c_{y}\end{bmatrix}$

$c_θ$= damping associated with beam rotation

$c_y$= damping associated with motor vertical motion

For small damping (typical of this setup), off-diagonal terms are negligible.

Solve the eigenvalue problem for the undamped system:

$\text{det}(K - \omega^2 M) = 0$

Substituting M and K gives a quadratic in $\omega^2$.

Solving it gives $ω_1$ and $ω_2$, the first two natural frequencies (rad/s).

For each $ω_i$, solve:
$\left( \mathbf{K} - \omega_i^{2}\,\mathbf{M} \right)\boldsymbol{\Phi}_i = \mathbf{0}$

$Φ_i$= i-th mode shape vector

Normalize $Φ_i$

This gives $Φ_1$ and $Φ_2$.

For each mode i:

$\zeta_i = \frac{\boldsymbol{\Phi}_i^{\mathsf{T}} \mathbf{C} \boldsymbol{\Phi}_i}
{2\,\omega_i\,\boldsymbol{\Phi}_i^{\mathsf{T}} \mathbf{M} \boldsymbol{\Phi}_i}$

Step by step:

1. Compute modal mass:
$M_i = \Phi_i^T M \Phi_i$

2. Compute modal damping:
$C_i = \Phi_i^T C \Phi_i$

3. Compute damping ratio:
$\zeta_i = \frac{C_i}{2M_i\omega_i}$

This gives $ζ_1$ and $ζ_2$.


	​


	​







